<a href="https://colab.research.google.com/github/mralamdari/NLP-Text_Classification/blob/main/NLP_Text2Vec_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import os
import gzip
import nltk
import shutil
import gensim
import sklearn
import warnings
import pandas as pd
import matplotlib.pyplot as plt 

nltk.download('punkt')
nltk.download('stopwords')
warnings.filterwarnings('ignore')
from sklearn import metrics
from sklearn import linear_model
from sklearn import model_selection

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Doc2Vec

In [ ]:
!wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
!wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/test_data.csv
!ls -lah DATAPATH
path = "DATAPATH/train_data.csv"

In [3]:
df = pd.read_csv(path)
df.shape

(30000, 2)

In [4]:
df.tail(10)

,sentiment,content
29990,fun,finally get my hands on my laptop! just ate t...
29991,happiness,@twtboxdj Thanks Mr. DJ!
29992,love,@Oprah Happy mother's day Oprah. You're a mom...
29993,neutral,Just finished watching He's Just Not That Into...
29994,fun,Hanging with my cousin Jimmy then hopefully ha...
29995,happiness,I had a great date last night...tried to find ...
29996,sadness,With alex
29997,happiness,@fureousangel that is comedy good luck my fri...
29998,fun,stephs grad party gr8! shoved cake in her face...
29999,happiness,@jesfive SWEEEEET - San Fran is awesome!!!! L...


In [5]:
df.sentiment.value_counts()

worry         7433
neutral       6340
sadness       4828
happiness     2986
love          2068
surprise      1613
hate          1187
fun           1088
relief        1021
empty          659
enthusiasm     522
boredom        157
anger           98
Name: sentiment, dtype: int64

In [6]:
selected_features = ['worry', 'neutral', 'happiness']
sub_df = df[df.sentiment.isin(selected_features)]
sub_df

,sentiment,content
4,neutral,@dannycastillo We want to trade with someone w...
5,worry,Re-pinging @ghostridah14: why didn't you go to...
7,worry,Hmmm. http://www.djhero.com/ is down
10,neutral,cant fall asleep
11,worry,Choked on her retainers
...,...,...
29991,happiness,@twtboxdj Thanks Mr. DJ!
29993,neutral,Just finished watching He's Just Not That Into...
29995,happiness,I had a great date last night...tried to find ...
29997,happiness,@fureousangel that is comedy good luck my fri...


In [7]:
tweeter = nltk.tokenize.TweetTokenizer(strip_handles=True, preserve_case=False)
stopwords = set(nltk.corpus.stopwords.words('english'))

stopwords_digits_remove = lambda tokens: [token for token in tokens if token not in stopwords and not token.isdigit()]
preprocess = lambda doc: [stopwords_digits_remove(tweeter.tokenize(content)) for content in doc]

mydata = preprocess(sub_df.content)
mycats = sub_df.sentiment

In [8]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(mydata, mycats, random_state=324)

In [9]:
train_doc2vec = [gensim.models.doc2vec.TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(x_train)]

In [10]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, alpha=0.025, dm=1, epochs=100)
model.build_vocab(train_doc2vec)

In [11]:
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)

In [12]:
train_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in x_train]
test_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in x_test]

In [14]:
np.array(train_vectors).shape, np.array(test_vectors).shape

((12569, 50), (4190, 50))

In [15]:
train_vectors[0]

array([-0.03098557, -0.18006831, -0.02823433,  0.18769668,  0.04804472,
        0.00737272,  0.02521468, -0.04709544, -0.2512116 , -0.07990362,
        0.0777841 , -0.19804786, -0.19192266, -0.04381737, -0.00838095,
        0.18414299, -0.16065387, -0.01527427, -0.04631079,  0.12437838,
        0.00117282,  0.14441812, -0.03413702,  0.14301172, -0.16492637,
        0.15638602,  0.0515771 ,  0.1525753 ,  0.05061591,  0.00561301,
        0.08640645,  0.04258852,  0.05532967,  0.05309271,  0.18171619,
       -0.09388975, -0.16126896, -0.02302054,  0.04398725,  0.17610832,
        0.03199048, -0.10280087,  0.15037712,  0.03241836,  0.24931793,
       -0.04825146,  0.16324289, -0.1485301 , -0.12004814, -0.04575532],
      dtype=float32)

In [16]:
lr_model = linear_model.LogisticRegression(class_weight='balanced')
lr_model.fit(train_vectors, y_train)
preds = lr_model.predict(test_vectors)

In [17]:
print(metrics.classification_report(y_test, preds))

              precision    recall  f1-score   support

   happiness       0.36      0.50      0.42       761
     neutral       0.47      0.57      0.51      1597
       worry       0.60      0.40      0.48      1832

    accuracy                           0.48      4190
   macro avg       0.48      0.49      0.47      4190
weighted avg       0.51      0.48      0.48      4190



#FastText

In [ ]:
!wget -P DATAPATH https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gz
!tar -xvf DATAPATH/dbpedia_csv.tar.gz -C DATAPATH
!ls -lah DATAPATH
data_path = 'DATAPATH'

In [41]:
train_file = data_path + '/dbpedia_csv/train.csv'
df = pd.read_csv(train_file, header=None, names=['class','name','description'])
test_file = data_path + '/dbpedia_csv/test.csv'
df_test = pd.read_csv(test_file, header=None, names=['class','name','description'])
print("Train:{} Test:{}".format(df.shape,df_test.shape))

Train:(560000, 3) Test:(70000, 3)


In [44]:
class_dict={
            1:'Company',
            2:'EducationalInstitution',
            3:'Artist',
            4:'Athlete',
            5:'OfficeHolder',
            6:'MeanOfTransportation',
            7:'Building',
            8:'NaturalPlace',
            9:'Village',
            10:'Animal',
            11:'Plant',
            12:'Album',
            13:'Film',
            14:'WrittenWork'
        }

# Mapping the classes
df['class_name'] = df['class'].map(class_dict)
df.tail(10)

,class,name,description,class_name
559990,14,Endocrine Research,Endocrine Research is a peer-reviewed medical...,WrittenWork
559991,14,A Thousand Splendid Suns,A Thousand Splendid Suns is a 2007 novel by A...,WrittenWork
559992,14,Western People,The Western People is a weekly local newspape...,WrittenWork
559993,14,Mademoiselle (magazine),Mademoiselle was a women's magazine first pub...,WrittenWork
559994,14,Scoliosis (journal),Scoliosis (ISSN 1748-7161) is an open-access ...,WrittenWork
559995,14,Barking in Essex,Barking in Essex is a Black comedy play direc...,WrittenWork
559996,14,Science & Spirit,Science & Spirit is a discontinued American b...,WrittenWork
559997,14,The Blithedale Romance,The Blithedale Romance (1852) is Nathaniel Ha...,WrittenWork
559998,14,Razadarit Ayedawbon,Razadarit Ayedawbon (Burmese: ရာဇာဓိရာဇ် အရေး...,WrittenWork
559999,14,The Vinyl Cafe Notebooks,Vinyl Cafe Notebooks: a collection of essays ...,WrittenWork


In [47]:
def data_cleaner(text, normalize=True):
  s = str(text).replace(',',' ').replace('"','').replace('\'',' \' ').replace('.',' . ').replace('(',' ( ').replace(')',' ) ').replace('!',' ! ').replace('?',' ? ').replace(':',' ').replace(';',' ').lower()    
  if normalize:
    s = s.normalize('NFKD').str.encode('ascii','ignore').str.decode('utf-8')
  return s

In [60]:
def clean_df(data, cleanit= False, shuffle_it=False, encodeit=False, label_prefix='__class__'):
    df = data[['name','description']].copy(deep=True)
    df['class'] = label_prefix + data['class'].astype(str) + ' '
    
    if cleanit:
        df['name'] = df['name'].apply(lambda x: data_cleaner(x, encodeit))
        df['description'] = df['description'].apply(lambda x: data_cleaner(x, encodeit))

    if shuffle_it:
        df.sample(frac=1).reset_index(drop=True)
    return df

In [58]:
df_train_cleaned = clean_df(df, True, True)
df_test_cleaned = clean_df(df_test, True, True)

In [59]:
df_train_cleaned

,name,description,class
0,e . d . abbott ltd,abbott of farnham e d abbott limited was a br...,__class__1
1,schwan-stabilo,schwan-stabilo is a german maker of pens for ...,__class__1
2,q-workshop,q-workshop is a polish company located in poz...,__class__1
3,marvell software solutions israel,marvell software solutions israel known as ra...,__class__1
4,bergan mercy medical center,bergan mercy medical center is a hospital loc...,__class__1
...,...,...,...
559995,barking in essex,barking in essex is a black comedy play direc...,__class__14
559996,science & spirit,science & spirit is a discontinued american b...,__class__14
559997,the blithedale romance,the blithedale romance ( 1852 ) is nathanie...,__class__14
559998,razadarit ayedawbon,razadarit ayedawbon ( burmese ရာဇာဓိရာဇ် အရ...,__class__14


In [61]:
train_file = data_path + '/dbpedia_train.csv'
df_train_cleaned.to_csv(train_file, header=None, index=False, columns=['class','name','description'] )

test_file = data_path + '/dbpedia_test.csv'
df_test_cleaned.to_csv(test_file, header=None, index=False, columns=['class','name','description'] )

In [ ]:
# !pip install fasttext
import fasttext
from fasttext import train_supervised
model = train_supervised(input=train_file, label='__class__', lr=1.0, epoch=75, loss='ova', wordNgrams=2, dim=200, thread=2, verbose=100)

In [ ]:
for k in range(1, 6):
  results = model.test(test_file, k=k)
  print(f"Test Samples: {results[0]} Precision@{k} : {results[1]*100:2.4f} Recall@{k} : {results[2]*100:2.4f}")